# Regulier Tag Onderhoud

In [ ]:
%run ./__init__edwh__notebooks.ipynb

In [ ]:
omgeving_leiden= Tag("Omgeving Leiden")
omgeving_leiden_gid = omgeving_leiden.gid

In [ ]:
%%sql
select count(*)
  from tag 
 where (slug like  '%-leiden'
        or slug like '%-zoeterwoude'
        or slug like '%-voorschoten'
        or slug like '%-oegstgeest'
        or slug like '%-leiderdorp'
      ) and meta_tags like '%|e5b80b90-1851-42d9-aebc-87327e87ed09|%'


In [ ]:
# TOrganisations is een globale tag omdat deze gedefinieerd is in __init__edw__notebooks.ipynb
def organisatie_tags_in_gemeenten(*gemeenten):
    slug_query = db.tag.id < 0 #query die niets oplevert 
    for gemeente in gemeenten:
        # school namen hebben 
        slug_query |= db.tag.slug.like(f'%-{gemeente}')
    return db(slug_query & (db.tag.meta_tags.contains(TOrganisations.gid))).select()

In [ ]:
organisatie_tags = organisatie_tags_in_gemeenten('leiden','zoeterwoude','voorschoten','oegstgeest','leiderdorp')
organisatie_tags 

In [ ]:
def items_tagged_with(tag):
    active_items = db.item.author != None
    active_items &=db.item.platform == 'SvS'
    selected_tag = db.item.tags.contains(tag.gid)
    return db(active_items & selected_tag).select()


In [ ]:
items_tagged_with(omgeving_leiden)

In [ ]:
%%sql items << 
select item.id as id, item.gid as gid, item.name as name, u.name as author, tags, ts_changed
  from item inner join "user" u  on item.author = u.gid
 where item.author is not null 
   and item.platform = 'SvS'
   and item.tags like '%|{omgeving_leiden_gid}|%'

In [ ]:
# converteer resultset naar een dataframe zodat we hier mee kunnen doorwerken
items = items.DataFrame()

In [ ]:
# items['tags'] = items.progress_apply(lambda x: '\n'.join([Tag(_).name for _ in x['tags'].split('|') if _.strip()]), axis=1)
# items

In [ ]:
for idx, item in items.iterrows():
    # display(Markdown(f"{item.id}|{item.gid}|{item.name}||{item.author:>20}|{item.tags}|{item.ts_changed}"))
    tag_names = sorted(handig.names([Tag(_) for _ in item.tags.split('|') if _]))
    
    display(Markdown(f"## #{idx+1} - {item['name']} - *{item.author.strip()}*"))
    display(Markdown(f"  * gid: `{item.gid}`\n  * tags: "+', '.join(f'`{_}`' for _ in tag_names)+f'\n  * changed: {item.ts_changed}'))
    display(Markdown(f""))
    